In [1]:
import AdventOfCode

In [2]:
import qualified Data.Vector as V

In [3]:
input <- dayLines 19

In [4]:
import Data.Bits

addr, addi :: V.Vector Int -> V.Vector Int -> V.Vector Int

oprr op rs is = let
    opA = rs V.! (is V.! 0)
    opB = rs V.! (is V.! 1)
    rsC = op opA opB
    in rs V.// [(is V.! 2, rsC)]

opri op rs is = let
    opA = rs V.! (is V.! 0)
    opB = (is V.! 1)
    rsC = op opA opB
    in rs V.// [(is V.! 2, rsC)]

opir op rs is = let
    opA = (is V.! 0)
    opB = rs V.! (is V.! 1)
    rsC = op opA opB
    in rs V.// [(is V.! 2, rsC)]

fromBool False = 0
fromBool True = 1

(addr, addi) = (oprr (+), opri (+))
(mulr, muli) = (oprr (*), opri (*))
(banr, bani) = (oprr (fromIntegral .: (.&.)), opri (.&.))
(borr, bori) = (oprr (fromIntegral .: (.|.)), opri (.|.))
(setr, seti) = (oprr const, opir const)
(gtrr, gtri, gtir) = (oprr (fromBool .: (>)), opri (fromBool .: (>)), opir (fromBool .: (>)))
(eqrr, eqri, eqir) = (oprr (fromBool .: (==)), opri (fromBool .: (==)), opir (fromBool .: (==)))

In [5]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as C
import Data.Functor (($>))

parseInstr :: C.Parser (String, V.Vector Int -> V.Vector Int -> V.Vector Int)
parseInstr = C.choice
    [ "addr" $> ("addr",addr)
    , "addi" $> ("addi",addi)
    , "mulr" $> ("mulr",mulr)
    , "muli" $> ("muli",muli)
    , "banr" $> ("banr",banr)
    , "bani" $> ("bani",bani)
    , "borr" $> ("borr",borr)
    , "bori" $> ("bori",bori)
    , "setr" $> ("setr",setr)
    , "seti" $> ("seti",seti)
    , "gtrr" $> ("gtrr",gtrr)
    , "gtri" $> ("gtri",gtri)
    , "gtir" $> ("gtir",gtir)
    , "eqrr" $> ("eqrr",eqrr)
    , "eqri" $> ("eqri",eqri)
    , "eqir" $> ("eqir",eqir)
    ]

In [10]:
parseLine :: C.Parser ((V.Vector Int -> V.Vector Int -> V.Vector Int), String, V.Vector Int)
parseLine = do
    (name,instr) <- parseInstr
    C.skipSpace
    a <- C.decimal
    C.skipSpace
    b <- C.decimal
    C.skipSpace
    c <- C.decimal
    pure (instr, name, V.fromList [a,b,c])

In [11]:
instructions = V.fromList $ map (parsed parseLine) $ drop 1 input
:t instructions

instructions :: Vector (Vector Int -> Vector Int -> Vector Int, String, Vector Int)

In [12]:
import Debug.Trace

loop rs instrs = let
    ip = (rs V.! 3)
    in if ip < 0 || ip > V.length instrs
        then rs
        else let
            (instr,name,is) = instrs V.! ip
            rs' = traceShow (name,is) $ instr rs is
            ip' = rs' V.! 3
            rs'' = rs' V.// [(3,ip'+1)]
            in loop rs'' instrs

rs = V.fromList $ replicate 6 0
loop rs instructions

[2106,1,954,257,955,955]

In [9]:
-- loop (V.fromList [1, 0, 0, 0, 0, 0]) instructions